# BOT AMAZON web scraping

## COMPILADO

### V1

In [7]:
import requests  # Biblioteca para fazer requisições HTTP, usada para acessar as páginas da Amazon
from lxml import html  # Biblioteca para parsing de HTML, usada para extrair informações de páginas
import pandas as pd  # Biblioteca para manipulação de dados, usada para criar e manipular DataFrames
import time  # Biblioteca para gerenciar o tempo, usada para controlar intervalos entre requisições
import re  # Biblioteca de expressões regulares, usada para manipulação de strings
from datetime import datetime  # Biblioteca para manipular datas, usada para adicionar data aos dados
import logging  # Biblioteca para criação de logs, usada para registrar eventos e erros
import random  # Biblioteca para gerar números aleatórios, usada para intervalos aleatórios entre requisições
import os  # Biblioteca para manipulação de sistemas operacionais, usada para criar diretórios

# Diretório para salvar os logs
log_directory = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/logs/"
os.makedirs(log_directory, exist_ok=True)  # Cria o diretório de logs se ele ainda não existir

# Configuração do logging para salvar no diretório especificado
logging.basicConfig(
    filename=os.path.join(log_directory, 'amazon_scraper.log'),  # Define o nome e local do arquivo de log
    level=logging.INFO,  # Define o nível de log (INFO), para registrar eventos gerais
    format='%(asctime)s - %(levelname)s - %(message)s'  # Define o formato da mensagem de log: data/hora, nível e mensagem
)

# Função para obter os links das categorias da página inicial (Elektronica, Software ou Boeken)
def get_category_links(url):
    """
    Faz uma requisição à página principal e extrai os links das categorias.
    
    Parâmetros:
        url (str): URL da página principal da Amazon (ex: Best Sellers)

    Retorno:
        list: Lista de dicionários contendo o nome e o link completo de cada categoria
    """
    # Define o cabeçalho da requisição para simular um navegador e evitar bloqueios
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    try:
        # Faz a requisição à URL com o cabeçalho definido
        response = requests.get(url, headers=headers, timeout=10)  # Timeout de 10 segundos para evitar travamentos
        response.raise_for_status()  # Gera um erro se o status da requisição for diferente de 200 (OK)
    except requests.exceptions.RequestException as e:
        logging.error(f"Erro ao acessar a URL {url}: {e}")  # Registra o erro nos logs
        return []  # Retorna uma lista vazia caso haja falha na requisição

    # Faz o parsing do conteúdo HTML da página
    tree = html.fromstring(response.content)
    
    # Usa XPath para encontrar os links das categorias
    categories = tree.xpath('//div[contains(@class, "_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz")]//a')
    
    # Cria uma lista de dicionários contendo o nome e o link completo de cada categoria
    category_links = [{'category': cat.text_content().strip(), 'link': 'https://www.amazon.nl' + cat.get('href')} for cat in categories]

    return category_links  # Retorna a lista de categorias

# Função para extrair detalhes dos produtos de uma categoria específica
def extract_product_details(items, category):
    """
    Extrai detalhes dos produtos de uma categoria (ex: Best Sellers, Movers and Shakers).

    Parâmetros:
        items (list): Lista de elementos de produtos extraídos da página da categoria
        category (str): Nome da categoria de produtos

    Retorno:
        list: Lista de dicionários contendo os detalhes de cada produto
    """
    products = []  # Lista onde serão armazenados os detalhes dos produtos
    today_date = datetime.today().strftime('%Y-%m-%d')  # Data atual formatada para ser usada em cada produto

    # Itera sobre cada item da lista de produtos
    for index, item in enumerate(items, start=1):
        # Extrai o ID do produto (ASIN)
        product_id = item.xpath('.//@data-asin')
        product_id = product_id[0] if product_id else "No ID"  # Usa "No ID" caso o ID não seja encontrado

        # Extrai a posição no ranking (por exemplo, #1, #2, etc.)
        position = item.xpath('.//span[@class="zg-bdg-text"]/text()')
        position = position[0].strip() if position else str(index)  # Se a posição não estiver disponível, usa o índice do loop

        # Extrai o link da imagem do produto
        image = item.xpath('.//img[contains(@class, "a-dynamic-image")]/@src')
        image_link = image[0] if image else "No image link"  # Caso não haja imagem, usa "No image link"

        # Extrai o título do produto
        title = item.xpath('.//a/span/div/text()')
        title = title[0].strip() if title else "No title"  # Se não houver título, usa "No title"

        # Extrai o link do produto
        link = item.xpath('.//a[contains(@class, "a-link-normal")]/@href')
        product_link = "https://www.amazon.nl" + link[0] if link else "No product link"

        # Extrai o nome do produto (parte da URL)
        name = item.xpath('.//a[@class="a-link-normal aok-block"]/@href')
        name = name[0].split('/')[1] if name else "No name"

        # Extrai a classificação/avaliação do produto
        rating = item.xpath('.//span[contains(@class, "a-icon-alt")]/text()')
        rating = rating[0].strip() if rating else "No rating"

        # Extrai o número de avaliações (quantidade de reviews)
        reviews = item.xpath('.//span[@class="a-size-small"]/text()')
        reviews = reviews[0].strip() if reviews else "No reviews"

        # Extrai o preço do produto
        price = item.xpath('.//span[contains(@class, "p13n-sc-price")]/text()')
        if price:
            price = price[0].strip()
            currency_symbol = ''.join(re.findall(r'[^\d.,]', price))  # Extrai o símbolo da moeda
            value = ''.join(re.findall(r'[\d.,]+', price))  # Extrai o valor numérico do preço
        else:
            currency_symbol = "Not Available"
            value = "Not Available"

        # Adiciona os detalhes do produto à lista de produtos
        products.append({
            "category": category,
            "rank": position,
            "asin": product_id,
            "name": name,
            "title": title,
            "rating": rating,
            "reviews": reviews,
            "symbol": currency_symbol,
            "value": value,
            "image": image_link,
            "link": product_link,        
            "date": today_date
        })

    return products  # Retorna a lista de produtos

# Função que obtém os produtos mais vendidos em uma categoria específica, com retries para evitar falhas temporárias
def get_amazon_bestsellers(url, category, retries=5, backoff_factor=0.3):
    """
    Faz a requisição para uma categoria específica da Amazon e coleta os produtos. Tenta novamente em caso de falhas.

    Parâmetros:
        url (str): URL da categoria da Amazon
        category (str): Nome da categoria de produtos (Best Sellers, Movers and Shakers, etc.)
        retries (int): Número máximo de tentativas de requisição
        backoff_factor (float): Fator de tempo para espera entre retries

    Retorno:
        list: Lista de produtos da categoria, ou lista vazia em caso de falha
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # Loop para realizar retries em caso de falha na requisição
    for i in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Faz a requisição com um timeout de 10 segundos
            response.raise_for_status()  # Gera um erro se o status não for 200 (OK)
            break  # Se a requisição for bem-sucedida, sai do loop
        except requests.exceptions.RequestException as e:
            logging.error(f"Erro ao acessar {url} na tentativa {i+1}/{retries}: {e}")  # Registra o erro nos logs
            time.sleep(backoff_factor * (2 ** i))  # Aumenta o tempo de espera exponencialmente (backoff)
        if i == retries - 1:
            return []  # Retorna lista vazia após o número máximo de retries

    # Faz o parsing do conteúdo da página
    tree = html.fromstring(response.content)
    
    # Extrai os produtos da categoria com base no identificador "p13n-asin-index"
    items = tree.xpath('//div[contains(@id, "p13n-asin-index")]')
    logging.info(f"Encontrados {len(items)} itens na categoria {category}.")

    # Retorna os detalhes dos produtos extraídos
    return extract_product_details(items, category)

# Função para salvar os dados consolidados em Excel
def save_to_excel_consolidated(products, directory_path):
    """
    Salva os produtos extraídos em um único arquivo Excel consolidado.

    Parâmetros:
        products (list): Lista de produtos de todas as categorias
        directory_path (str): Caminho para salvar o arquivo Excel
    """
    if not products:
        logging.warning("Nenhum produto encontrado nas categorias.")  # Registra um aviso se não houver produtos
        return

    # Cria um DataFrame a partir da lista de produtos
    df = pd.DataFrame(products)
    
    # Define o nome do arquivo com base na data atual
    today_date = datetime.today().strftime('%Y-%m-%d')
    filename = f"{directory_path}/extract_amazon_full_TESTE{today_date}.xlsx"
    
    # Salva o DataFrame em um arquivo Excel
    df.to_excel(filename, index=False)
    logging.info(f"Dados consolidados salvos em {filename}")  # Registra a conclusão da tarefa

# Função que processa uma categoria específica e adiciona a coluna "origin"
def process_category(category_url, category_type):
    """
    Processa uma categoria específica, extrai os produtos e adiciona uma coluna "origin" para identificar a origem.

    Parâmetros:
        category_url (str): URL da categoria
        category_type (str): Tipo de categoria (ex: Best Sellers, Movers and Shakers)

    Retorno:
        list: Lista de produtos com a coluna "origin" indicando a origem dos dados
    """
    # Obtém os links das subcategorias
    category_links = get_category_links(category_url)
    all_products = []

    # Processa cada subcategoria
    for category_link in category_links:
        category = category_link['category']  # Nome da subcategoria
        link = category_link['link']  # Link da subcategoria
        logging.info(f"Processando categoria: {category}")
        
        # Extrai os produtos da subcategoria
        products = get_amazon_bestsellers(link, category)
        
        # Adiciona a origem dos dados na coluna "origin"
        for product in products:
            product['origin'] = category_type  # Adiciona a origem (ex: best_sellers)
        
        all_products.extend(products)  # Adiciona os produtos à lista geral
        
        # Pausa com intervalo aleatório para evitar sobrecarga no servidor
        time.sleep(random.uniform(0, 1))

    return all_products  # Retorna a lista de produtos processados

# Função principal que coordena todo o processo
def extract_data():
    directory_path = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/output/bot_amazon"

    # Lista para armazenar os produtos de todas as categorias
    all_products = []

    # Processa Best Sellers e adiciona à lista consolidada
    base_url_bestsellers = "https://www.amazon.nl/gp/bestsellers/"
    all_products.extend(process_category(base_url_bestsellers, "best_sellers"))

    # Processa Movers and Shakers e adiciona à lista consolidada
    #base_url_movers_shakers = "https://www.amazon.nl/gp/movers-and-shakers/"
    #all_products.extend(process_category(base_url_movers_shakers, "movers_and_shakers"))

    # Processa New Releases e adiciona à lista consolidada
    #base_url_new_releases = "https://www.amazon.nl/gp/new-releases/"
    #all_products.extend(process_category(base_url_new_releases, "new_releases"))

    # Cria um DataFrame com todos os produtos
    df_consolidated = pd.DataFrame(all_products)
    df_consolidated['reviews'] = df_consolidated['reviews'].apply(lambda x: re.sub(r'[.,]', '', str(x)))
    df_consolidated['reviews'] = df_consolidated['reviews'].replace("No reviews", 0)  # Substituir "No reviews" por 0
    df_consolidated['reviews'] = df_consolidated['reviews'].astype(int)

    # Exibe o DataFrame final para visualização
    #print(df_final)

    # Salva todos os dados consolidados em um arquivo Excel único
    save_to_excel_consolidated(all_products, directory_path)

    logging.info("Processo finalizado!")  # Registra a finalização do processo

    return df_consolidated

if __name__ == "__main__":
    extract_data()


KeyboardInterrupt: 

### V2

In [9]:
import requests  # Biblioteca para fazer requisições HTTP, usada para acessar as páginas da Amazon
from lxml import html  # Biblioteca para parsing de HTML, usada para extrair informações de páginas
import pandas as pd  # Biblioteca para manipulação de dados, usada para criar e manipular DataFrames
import time  # Biblioteca para gerenciar o tempo, usada para controlar intervalos entre requisições
import re  # Biblioteca de expressões regulares, usada para manipulação de strings
from datetime import datetime  # Biblioteca para manipular datas, usada para adicionar data aos dados
import logging  # Biblioteca para criação de logs, usada para registrar eventos e erros
import random  # Biblioteca para gerar números aleatórios, usada para intervalos aleatórios entre requisições
import os  # Biblioteca para manipulação de sistemas operacionais, usada para criar diretórios

# Diretório para salvar os logs
log_directory = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/logs/"
os.makedirs(log_directory, exist_ok=True)  # Cria o diretório de logs se ele ainda não existir

# Configuração do logging para salvar no diretório especificado
logging.basicConfig(
    filename=os.path.join(log_directory, 'amazon_scraper.log'),  # Define o nome e local do arquivo de log
    level=logging.INFO,  # Define o nível de log (INFO), para registrar eventos gerais
    format='%(asctime)s - %(levelname)s - %(message)s'  # Define o formato da mensagem de log: data/hora, nível e mensagem
)

logging.info(f"Extração iniciada")  

# Função para obter os links das categorias da página inicial (Elektronica, Software ou Boeken)
def get_category_links(url, retries=5, backoff_factor=1):
    """
    Faz uma requisição à página principal e extrai os links das categorias.
    Se falhar em encontrar os links, tenta novamente até 5 vezes.

    Parâmetros:
        url (str): URL da página principal da Amazon (ex: Best Sellers)
        retries (int): Número máximo de tentativas de requisição
        backoff_factor (float): Fator de tempo para espera entre retries

    Retorno:
        list: Lista de dicionários contendo o nome e o link completo de cada categoria
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    for attempt in range(retries):
        try:
            # Faz a requisição à URL com o cabeçalho definido
            response = requests.get(url, headers=headers, timeout=10)  # Timeout de 10 segundos para evitar travamentos
            response.raise_for_status()  # Gera um erro se o status da requisição for diferente de 200 (OK)
            
            # Faz o parsing do conteúdo HTML da página
            tree = html.fromstring(response.content)
            
            # Usa XPath para encontrar os links das categorias
            categories = tree.xpath('//div[contains(@class, "_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz")]//a')
            
            if categories:
                # Cria uma lista de dicionários contendo o nome e o link completo de cada categoria
                category_links = [{'category': cat.text_content().strip(), 'link': 'https://www.amazon.nl' + cat.get('href')} for cat in categories]
                logging.info(f"Links de subcategorias encontrados para a URL: {url}")
                return category_links  # Retorna a lista de categorias
            else:
                # Se não encontrou categorias, tenta novamente
                logging.warning(f"Nenhuma subcategoria encontrada na tentativa {attempt+1}/{retries} para a URL: {url}")
                time.sleep(backoff_factor * (2 ** attempt))  # Espera exponencialmente mais a cada tentativa
        
        except requests.exceptions.RequestException as e:
            logging.error(f"Erro ao acessar a URL {url} na tentativa {attempt+1}/{retries}: {e}")
            time.sleep(backoff_factor * (2 ** attempt))  # Espera exponencialmente mais a cada tentativa
    
    logging.error(f"Não foram encontrados links de subcategorias para a URL: {url} após {retries} tentativas.")
    return []  # Retorna uma lista vazia caso haja falha na requisição após o número máximo de tentativas


# Função para extrair detalhes dos produtos de uma categoria específica
import re  # Biblioteca para expressões regulares

def extract_product_details(items, category):
    """
    Extrai detalhes dos produtos de uma categoria (ex: Best Sellers, Movers and Shakers).

    Parâmetros:
        items (list): Lista de elementos de produtos extraídos da página da categoria
        category (str): Nome da categoria de produtos

    Retorno:
        list: Lista de dicionários contendo os detalhes de cada produto
    """
    products = []  # Lista onde serão armazenados os detalhes dos produtos
    today_date = datetime.today().strftime('%Y-%m-%d')  # Data atual formatada para ser usada em cada produto

    # Itera sobre cada item da lista de produtos
    for index, item in enumerate(items, start=1):
        # Extrai o ID do produto (ASIN)
        product_id = item.xpath('.//@data-asin')
        product_id = product_id[0] if product_id else "No ID"  # Usa "No ID" caso o ID não seja encontrado

        # Extrai a posição no ranking (por exemplo, #1, #2, etc.)
        position = item.xpath('.//span[@class="zg-bdg-text"]/text()')
        position = position[0].strip() if position else str(index)  # Se a posição não estiver disponível, usa o índice do loop

        # Extrai o link da imagem do produto
        image = item.xpath('.//img[contains(@class, "a-dynamic-image")]/@src')
        image_link = image[0] if image else "No image link"  # Caso não haja imagem, usa "No image link"

        # Extrai o título do produto
        title = item.xpath('.//a/span/div/text()')
        title = title[0].strip() if title else "No title"  # Se não houver título, usa "No title"

        # Extrai o link do produto
        link = item.xpath('.//a[contains(@class, "a-link-normal")]/@href')
        product_link = "https://www.amazon.nl" + link[0] if link else "No product link"

        # Extrai o nome do produto (parte da URL)
        name = item.xpath('.//a[@class="a-link-normal aok-block"]/@href')
        name = name[0].split('/')[1] if name else "No name"

        # Extrai a classificação/avaliação do produto
        rating = item.xpath('.//span[contains(@class, "a-icon-alt")]/text()')
        rating = rating[0].strip() if rating else "No rating"

        # Extrai o número de avaliações (quantidade de reviews)
        reviews = item.xpath('.//span[@class="a-size-small"]/text()')
        reviews = reviews[0].strip() if reviews else "No reviews"

        # Extrai o preço do produto
        price = item.xpath('.//span[contains(@class, "p13n-sc-price")]/text()')
        if price:
            price = price[0].strip()
            currency_symbol = ''.join(re.findall(r'[^\d.,]', price))  # Extrai o símbolo da moeda
            value = ''.join(re.findall(r'[\d.,]+', price))  # Extrai o valor numérico do preço
        else:
            currency_symbol = "Not Available"
            value = "Not Available"

        # Extrai o número do percentual de alta (por exemplo, "427%")
        perc_alta = item.xpath('.//span[contains(@class, "zg-carousel-pct-change")]/text()')
        if perc_alta:
            perc_alta = re.search(r'\d+', perc_alta[0].strip()).group(0)
        else:
            perc_alta = 0

        # Extrai a classificação de vendas completa (exemplo: "Classificação de vendas: 180 (anterior: 950)")
        classificacao_vendas = item.xpath('.//span[contains(@class, "zg-carousel-sales-movement")]/text()')
        classificacao_vendas = classificacao_vendas[0].strip() if classificacao_vendas else "No classificacao vendas"

        # Extrai apenas o número da classificação de vendas (antes dos parênteses) - (por exemplo, "180")
        match_vendas = re.search(r'(\d+)', classificacao_vendas)
        classificacao_vendas_numero = match_vendas.group(1) if match_vendas else 0

        # Extrai apenas o número da classificação de vendas anterior (dentro dos parênteses) - (por exemplo, "950")
        match_vendas_anterior = re.search(r'\((\d+)\)', classificacao_vendas)
        classificacao_vendas_anterior = match_vendas_anterior.group(1) if match_vendas_anterior else 0

        # Adiciona os detalhes do produto à lista de produtos
        products.append({
            "category": category,
            "rank": position,
            "asin": product_id,
            "name": name,
            "title": title,
            "rating": rating,
            "reviews": reviews,
            "symbol": currency_symbol,
            "value": value,
            "image": image_link,
            "link": product_link,
            "perc_alta": perc_alta,  # Apenas o número do percentual de alta
            "classificacao_vendas": classificacao_vendas_numero,  # Apenas o número da classificação de vendas
            "classificacao_vendas_anterior": classificacao_vendas_anterior,  # Apenas o número da classificação de vendas anterior
            "date": today_date
        })

    return products  # Retorna a lista de produtos

# Função que obtém os produtos mais vendidos em uma categoria específica, com retries para evitar falhas temporárias
def get_amazon_bestsellers(url, category, retries=5, backoff_factor=1):
    """
    Faz a requisição para uma categoria específica da Amazon e coleta os produtos. Tenta novamente em caso de falhas ou
    caso nenhum produto seja encontrado (len(items) == 0).

    Parâmetros:
        url (str): URL da categoria da Amazon
        category (str): Nome da categoria de produtos (Best Sellers, Movers and Shakers, etc.)
        retries (int): Número máximo de tentativas de requisição
        backoff_factor (float): Fator de tempo para espera entre retries

    Retorno:
        list: Lista de produtos da categoria, ou lista vazia em caso de falha
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # Loop para realizar retries em caso de falha na requisição ou no parsing dos itens
    for i in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Faz a requisição com um timeout de 10 segundos
            response.raise_for_status()  # Gera um erro se o status não for 200 (OK)
            
            # Faz o parsing do conteúdo da página
            tree = html.fromstring(response.content)
            
            # Extrai os produtos da categoria com base no identificador "p13n-asin-index"
            items = tree.xpath('//div[contains(@id, "p13n-asin-index")]')
            
            # Verifica se foram encontrados itens
            if len(items) > 0:
                logging.info(f"Encontrados {len(items)} itens na categoria {category}.")
                print(f"Encontrados {len(items)} itens na categoria {category}.")
                return extract_product_details(items, category)
            else:
                logging.warning(f"Nenhum item encontrado na categoria {category} na tentativa {i+1}/{retries}.")
                print(f"Nenhum item encontrado na categoria {category} na tentativa {i+1}/{retries}.")
                time.sleep(backoff_factor * (2 ** i))  # Aumenta o tempo de espera exponencialmente (backoff)
        
        except requests.exceptions.RequestException as e:
            logging.error(f"Erro ao acessar {url} na tentativa {i+1}/{retries}: {e}")  # Registra o erro nos logs
            time.sleep(backoff_factor * (2 ** i))  # Aumenta o tempo de espera exponencialmente (backoff)
        
        # Se o número máximo de tentativas for atingido e não houver itens, retorna uma lista vazia
        if i == retries - 1:
            logging.error(f"Falha ao obter produtos da categoria {category} após {retries} tentativas.")
            print(f"Falha ao obter produtos da categoria {category} após {retries} tentativas.")
            return []  # Retorna lista vazia após o número máximo de retries

# Função para salvar os dados consolidados em Excel
def save_to_excel_consolidated(products, directory_path):
    """
    Salva os produtos extraídos em um único arquivo Excel consolidado.

    Parâmetros:
        products (list): Lista de produtos de todas as categorias
        directory_path (str): Caminho para salvar o arquivo Excel
    """
    if not products:
        logging.warning("Nenhum produto encontrado nas categorias.")  # Registra um aviso se não houver produtos
        print("Nenhum produto encontrado nas categorias.")
        return

    # Cria um DataFrame a partir da lista de produtos
    df = pd.DataFrame(products)
    
    # Define o nome do arquivo com base na data atual
    today_date = datetime.today().strftime('%Y-%m-%d')
    filename = f"{directory_path}/extract_amazon_full_{today_date}.xlsx"
    
    # Salva o DataFrame em um arquivo Excel
    df.to_excel(filename, index=False)
    logging.info(f"Dados consolidados salvos em {filename}")  # Registra a conclusão da tarefa

# Função que processa uma categoria específica e adiciona a coluna "origin"
def process_category(category_url, category_type):
    """
    Processa uma categoria específica, extrai os produtos e adiciona uma coluna "origin" para identificar a origem.

    Parâmetros:
        category_url (str): URL da categoria
        category_type (str): Tipo de categoria (ex: Best Sellers, Movers and Shakers)

    Retorno:
        list: Lista de produtos com a coluna "origin" indicando a origem dos dados
    """
    # Obtém os links das subcategorias
    category_links = get_category_links(category_url)
    
    if not category_links:
        logging.error(f"Não foram encontrados links de subcategorias para a URL: {category_url}")
        print(f"Não foram encontrados links de subcategorias para a URL: {category_url}")
        return []

    all_products = []

    # Processa cada subcategoria
    for category_link in category_links:
        category = category_link['category']  # Nome da subcategoria
        link = category_link['link']  # Link da subcategoria
        logging.info(f"Processando subcategoria: {category} - {category_type}")
        print(f"Processando subcategoria: {category} - {category_type}")
        
        # Extrai os produtos da subcategoria
        products = get_amazon_bestsellers(link, category)
        
        if not products:
            logging.warning(f"Nenhum produto encontrado na subcategoria: {category} - {category_type}")
        
        # Adiciona a origem dos dados na coluna "origin"
        for product in products:
            product['origin'] = category_type  # Adiciona a origem (ex: best_sellers)
        
        all_products.extend(products)  # Adiciona os produtos à lista geral
        
        # Pausa com intervalo aleatório para evitar sobrecarga no servidor
        time.sleep(random.uniform(10, 20))

    return all_products  # Retorna a lista de produtos processados

# Função principal que coordena todo o processo
def extract_data():
    print("Extração iniciada")

    directory_path = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/output/bot_amazon"

    # Lista para armazenar os produtos de todas as categorias
    all_products = []

    # Processa Best Sellers e adiciona à lista consolidada
    logging.info("Iniciando processamento de Best Sellers")
    base_url_bestsellers = "https://www.amazon.nl/gp/bestsellers/"
    best_sellers_products = process_category(base_url_bestsellers, "best_sellers")
    all_products.extend(best_sellers_products)
    logging.info(f"Produtos de Best Sellers: {len(best_sellers_products)}")

    # Pausa com intervalo aleatório para evitar sobrecarga no servidor
    time.sleep(random.uniform(10, 20))

    # Processa Movers and Shakers e adiciona à lista consolidada
    logging.info("Iniciando processamento de Movers and Shakers")
    base_url_movers_shakers = "https://www.amazon.nl/gp/movers-and-shakers/"
    movers_shakers_products = process_category(base_url_movers_shakers, "movers_and_shakers")
    all_products.extend(movers_shakers_products)
    logging.info(f"Produtos de Movers and Shakers: {len(movers_shakers_products)}")

    # Pausa com intervalo aleatório para evitar sobrecarga no servidor
    time.sleep(random.uniform(10, 20))

    # Processa New Releases e adiciona à lista consolidada
    logging.info("Iniciando processamento de New Releases")
    base_url_new_releases = "https://www.amazon.nl/gp/new-releases/"
    new_releases_products = process_category(base_url_new_releases, "new_releases")
    all_products.extend(new_releases_products)
    logging.info(f"Produtos de New Releases: {len(new_releases_products)}")

    # Verifica se foram encontrados produtos
    if not all_products:
        logging.error("Nenhum produto encontrado para as categorias processadas.")
        print("Nenhum produto encontrado.")
        return

    # Cria um DataFrame com todos os produtos
    df_consolidated = pd.DataFrame(all_products)
    
    # Limpeza e formatação de reviews
    df_consolidated['reviews'] = df_consolidated['reviews'].apply(lambda x: re.sub(r'[.,]', '', str(x)))
    df_consolidated['reviews'] = df_consolidated['reviews'].replace("No reviews", 0)  # Substituir "No reviews" por 0
    df_consolidated['reviews'] = df_consolidated['reviews'].astype(int)

    # Salva todos os dados consolidados em um arquivo Excel único
    save_to_excel_consolidated(all_products, directory_path)

    logging.info("Processo finalizado!")  # Registra a finalização do processo
    print("Base excel bruta salva")

    return df_consolidated

if __name__ == "__main__":
    extract_data()
print("Finalizado")

Extração iniciada
Processando subcategoria: Amazon Renewed - best_sellers
Encontrados 30 itens na categoria Amazon Renewed.
Processando subcategoria: Amazon-apparaten & accessoires - best_sellers
Encontrados 30 itens na categoria Amazon-apparaten & accessoires.
Processando subcategoria: Auto & motor - best_sellers
Encontrados 30 itens na categoria Auto & motor.
Processando subcategoria: Babyproducten - best_sellers
Encontrados 30 itens na categoria Babyproducten.
Processando subcategoria: Beauty - best_sellers
Encontrados 30 itens na categoria Beauty.
Processando subcategoria: Boeken - best_sellers
Encontrados 30 itens na categoria Boeken.
Processando subcategoria: Cadeaubonnen - best_sellers
Encontrados 30 itens na categoria Cadeaubonnen.
Processando subcategoria: Elektronica - best_sellers
Nenhum item encontrado na categoria Elektronica na tentativa 1/5.
Nenhum item encontrado na categoria Elektronica na tentativa 2/5.
Nenhum item encontrado na categoria Elektronica na tentativa 3/5.

## TRATAMENTO

In [63]:
# COMPILAR ARQUIVOS EM UM UNICO EXCEL

import os
import pandas as pd
import logging

# Configurar o log
log_file = 'C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/logs/compilacao_log.log'
logging.basicConfig(filename=log_file, level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Caminho da pasta de arquivos
folder_path = 'C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/output/bot_amazon/tratar'

# Listar todos os arquivos .xlsx na pasta
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# Inicializando uma lista para armazenar os DataFrames
df_list = []

# Variável para armazenar a estrutura de colunas de referência
reference_columns = None

# Iterar sobre os arquivos e verificar se todos têm o mesmo número de colunas
for file in files:
    file_path = os.path.join(folder_path, file)
    
    try:
        # Ler o arquivo Excel
        df = pd.read_excel(file_path)
        
        # Verificar colunas no primeiro arquivo
        if reference_columns is None:
            reference_columns = df.columns
        else:
            # Verificar se o número e os nomes das colunas coincidem com a referência
            if not df.columns.equals(reference_columns):
                logging.error(f'O arquivo {file} possui colunas diferentes: {df.columns}')
                continue  # Ignorar arquivos com colunas diferentes
        
        # Adicionar o DataFrame à lista
        df_list.append(df)
        logging.info(f'{file} compilado com sucesso.')
    
    except Exception as e:
        logging.error(f'Erro ao processar {file}: {str(e)}')

# Concatenar todos os DataFrames em um único
if df_list:
    df_combined = pd.concat(df_list, ignore_index=True)
    output_path = 'C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/output/bot_amazon/tratar_final.xlsx'
    
    # Salvar o arquivo compilado
    df_combined.to_excel(output_path, index=False)
    logging.info(f'Arquivo compilado salvo em {output_path}.')
else:
    logging.warning('Nenhum arquivo foi compilado.')

print(f'Compilação concluída. Verifique o log em {log_file} para detalhes.')


Compilação concluída. Verifique o log em C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/logs/compilacao_log.log para detalhes.


In [64]:
#LEITURA TRATAMENTO

import pandas as pd
import logging
import os


file_path = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/output/bot_amazon/tratar_final.xlsx"

def load_data(file_path):
    """Carrega os dados de um arquivo Excel."""
    try:
        df = pd.read_excel(file_path)
        logging.info(f"Dados carregados com sucesso de {file_path}")
        return df
    except Exception as e:
        logging.error(f"Erro ao carregar dados de {file_path}: {e}")
        return None
    
df = load_data(file_path)

In [65]:
# TRATAMENTO

import pandas as pd
import logging
import os
import numpy as np

# Diretório para salvar os logs
log_directory = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/logs/"
os.makedirs(log_directory, exist_ok=True)  # Cria o diretório de logs se ele ainda não existir

# Configuração do logging
logging.basicConfig(
    filename=os.path.join(log_directory, 'amazon_transform_data.log'),  # Define o nome e local do arquivo de log
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Tratamento iniciado")

def clean_data(df):
    """Limpa e formata os dados extraídos da Amazon."""
    try:
        # Tratamento da coluna 'rank'
        df['rank'] = df['rank'].str.replace('#', '', regex=False).astype(float)
        df['rank'] = df['rank'].fillna(0).astype(int)

        # Tratamento da coluna 'rating'
        df['rating'] = df['rating'].str.replace(' van 5 sterren', '').replace('No rating', "3").str.replace(",", ".")
        df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(3).round(2)

        # Tratamento da coluna 'reviews'
        df['reviews'] = df['reviews'].astype(str).str.replace('.', '', regex=False)  # Remove separadores de milhar
        df['reviews'] = pd.to_numeric(df['reviews'], errors='coerce').fillna(1).astype(int)

        # Tratamento da coluna 'value'
        df['value'] = pd.to_numeric(df['value'].str.replace(',', '.'), errors='coerce').fillna(999999).round(2)

        # Remover linhas com valor maior que 200
        df = df[df['value'] <= 200]

        # Filtra o DataFrame removendo categorias indesejadas
        df = df[~df['category'].isin(['Amazon Renewed', 'Amazon-apparaten & accessoires', 'Cadeaubonnen', 'Kindle Store'])]

        # Adicionar a coluna de moeda
        df['symbol'] = df['symbol'].str.replace('€', 'EUR')
        df = df.rename(columns={'symbol': 'currency'})


        logging.info('Dados limpos e valores inválidos removidos.')
        return df
    
    except Exception as e:
        logging.error(f"Erro durante o processo de limpeza de dados: {e}")
        raise e

def normalize_column(df, column):
    """Normaliza uma coluna usando min-max normalization."""
    try:
        min_val = df[column].min()
        max_val = df[column].max()
        if max_val != min_val:
            return (df[column] - min_val) / (max_val - min_val)
        else:
            return np.zeros(df.shape[0])  # Retorna uma coluna de zeros se min == max
    
    except Exception as e:
        logging.error(f"Erro ao normalizar a coluna {column}: {e}")
        raise e

def calculate_fields(df):
    """Adiciona campos normalizados e calcula o score e value_total."""
    try:
        # Normaliza as colunas
        df['normal_rating'] = normalize_column(df, 'rating')
        df['normal_reviews'] = normalize_column(df, 'reviews')
        df['normal_value'] = normalize_column(df, 'value')

        # Para o rank, normalizamos e invertemos a escala (menor rank = melhor)
        df['normal_rank'] = 1 - normalize_column(df, 'rank') # Inverte a normalização

        # Cálculo do score, agora com rank inversamente proporcional
        df['score'] = ((0.05 * df['normal_rating'] + 
                        0.3 * df['normal_reviews'] + 
                        0.5 * df['normal_rank'] +  # Utilizamos o rank invertido
                        0.15 * df['normal_value']) * 1000).astype(int)

        # Cálculo do value_total
        df['value_total'] = (df['value'] * df['reviews']).round(2)

        logging.info('Campos calculados adicionados com sucesso.')
        return df
    
    except Exception as e:
        logging.error(f"Erro ao calcular os campos: {e}")
        raise e


def transform_data(df_consolidated):
    """
    Executa o pipeline de transformação no DataFrame extraído (df_consolidated).
    
    Parâmetros:
        df_consolidated (DataFrame): DataFrame consolidado da extração.
    
    Retorno:
        DataFrame: DataFrame limpo e com campos calculados.
    """
    logging.info("Iniciando transformação")
    
    try:
        df_cleaned = clean_data(df_consolidated)
        df_transformed = calculate_fields(df_cleaned)
        logging.info("Transformação finalizada.")
        return df_transformed
    
    except Exception as e:
        logging.error(f"Erro na transformação de dados: {e}")
        raise e

# Exemplo de aplicação
df_transformed = transform_data(df)

# Analise descritiva

print(df_transformed.describe())
#print(df_transformed.describe(include='all'))
#print(df_transformed['rank'].value_counts())
print(df_transformed.isna().sum())



              rank       rating        reviews        value  perc_alta  \
count  8485.000000  8485.000000    8485.000000  8485.000000     8485.0   
mean     15.466352     4.129617    4996.111373    24.040255        0.0   
std       8.651400     0.705378   22347.579980    24.167918        0.0   
min       1.000000     1.000000       1.000000     0.010000        0.0   
25%       8.000000     3.800000       2.000000     9.990000        0.0   
50%      15.000000     4.400000      91.000000    16.620000        0.0   
75%      23.000000     4.600000    1600.000000    27.990000        0.0   
max      30.000000     5.000000  519358.000000   200.000000        0.0   

       classificacao_vendas  classificacao_vendas_anterior  normal_rating  \
count                8485.0                         8485.0    8485.000000   
mean                    0.0                            0.0       0.782404   
std                     0.0                            0.0       0.176345   
min                     0

## SALVAR EXCEL


In [66]:
from datetime import datetime
 # Define o nome do arquivo com base na data atual
directory_path = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/output/bot_amazon/final"
today_date = datetime.today().strftime('%Y-%m-%d')
filename = f"{directory_path}/base_amazon_final_13_09_24--17_09_24{today_date}.xlsx"
    
    # Salva o DataFrame em um arquivo Excel
df_transformed.to_excel(filename, index=False)



## CARGA POSTGRESQL

In [67]:
import pandas as pd

df = pd.read_excel("C:\\Users\\ThiagoBizacha\\Desktop\\Projeto_Automacao_Coleta_Dados\\data\\output\\bot_amazon\\final\\base_amazon_final_13_09_24--18_09_242024-09-27.xlsx")
print(df.shape)

(8485, 22)


In [68]:
##### CARGA FINAL POSTGRESQL

import psycopg2
import logging 
import os

def load_to_postgresql(df):
    """Carrega os dados do DataFrame no banco de dados PostgreSQL."""
    conn = psycopg2.connect(
        host="localhost",
        database="proj_dropshipping",
        user="postgres",
        password="admin"
    )
    logging.info(f"Conexão com o banco de dados realizada com sucesso!")  
    print("Conexão com o banco de dados realizada com sucesso!")

    cursor = conn.cursor()

    for index, row in df.iterrows():
        cursor.execute("""
            INSERT INTO public.amazon_nl_final (
                category, rank, asin, name, title, rating, reviews, currency, value, image, link, date, origin,  
                normal_rating, normal_reviews, normal_value, score, value_total, normal_rank, classificacao_vendas, 
                classificacao_vendas_anterior, perc_alta
            ) VALUES (%s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
            row['category'], row['rank'], row['asin'], row['name'], row['title'], row['rating'], row['reviews'],
            row['currency'], row['value'], row['image'], row['link'], row['date'], row['origin'], row['normal_rating'], 
            row['normal_reviews'], row['normal_value'], row['score'], row['value_total'], 
            row['normal_rank'], row['classificacao_vendas'], row['classificacao_vendas_anterior'], row['perc_alta']
        ))

    conn.commit()
    cursor.close()
    conn.close()
    logging.info(f"Dados carregados com sucesso no PostgreSQL - amazon_nl_final.") 
    print("Dados carregados com sucesso no PostgreSQL - amazon_nl_final.")
    
load_to_postgresql(df)

Conexão com o banco de dados realizada com sucesso!
Dados carregados com sucesso no PostgreSQL - amazon_nl_final.


In [69]:
### COMPILADO DADOS FINAIS

folder_path = 'C:\\Users\\ThiagoBizacha\\Desktop\\Projeto_Automacao_Coleta_Dados\\data\\output\\bot_amazon\\final'

def process_all_files_in_folder(folder_path):
    """Carrega todos os arquivos Excel da pasta e os carrega no PostgreSQL."""
    for file in os.listdir(folder_path):
        if file.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file)
            logging.info(f"Processando arquivo: {file_path}")
            print(f"Processando arquivo: {file_path}")

            try:
                # Ler o arquivo Excel
                df = pd.read_excel(file_path)
                logging.info(f"Arquivo {file} carregado com sucesso. Linhas: {df.shape[0]}")
                print(f"Arquivo {file} carregado com sucesso. Linhas: {df.shape[0]}")

                # Carregar o DataFrame no PostgreSQL
                load_to_postgresql(df)

            except Exception as e:
                logging.error(f"Erro ao processar o arquivo {file}: {str(e)}")
                print(f"Erro ao processar o arquivo {file}: {str(e)}")

# Executar o processamento de todos os arquivos
process_all_files_in_folder(folder_path)

Processando arquivo: C:\Users\ThiagoBizacha\Desktop\Projeto_Automacao_Coleta_Dados\data\output\bot_amazon\final\base_amazon_final_13_09_24--18_09_242024-09-27.xlsx
Arquivo base_amazon_final_13_09_24--18_09_242024-09-27.xlsx carregado com sucesso. Linhas: 8485
Conexão com o banco de dados realizada com sucesso!
Dados carregados com sucesso no PostgreSQL - amazon_nl_final.
Processando arquivo: C:\Users\ThiagoBizacha\Desktop\Projeto_Automacao_Coleta_Dados\data\output\bot_amazon\final\base_amazon_final_2024-09-19.xlsx
Arquivo base_amazon_final_2024-09-19.xlsx carregado com sucesso. Linhas: 1789
Conexão com o banco de dados realizada com sucesso!
Dados carregados com sucesso no PostgreSQL - amazon_nl_final.
Processando arquivo: C:\Users\ThiagoBizacha\Desktop\Projeto_Automacao_Coleta_Dados\data\output\bot_amazon\final\base_amazon_final_2024-09-20.xlsx
Arquivo base_amazon_final_2024-09-20.xlsx carregado com sucesso. Linhas: 1789
Conexão com o banco de dados realizada com sucesso!
Dados carreg

In [1]:
#//////////////////////// ##### CARGA BRUTA POSTGRESQL/////////////////////////////

import pandas as pd

df_bruto = pd.read_excel("C:\\Users\\ThiagoBizacha\\Desktop\\Projeto_Automacao_Coleta_Dados\\data\\output\\bot_amazon\\extract\\extract_amazon_full_2024-09-13.xlsx")
print(df_bruto.shape)

(1849, 16)


In [11]:
import os
from dotenv import load_dotenv

# Caminho absoluto do arquivo .env
# Obter o diretório atual de trabalho (onde o script está sendo executado)
project_root = os.getcwd()

# Combinar o caminho do diretório com o local do .env
dotenv_path = os.path.join(project_root, 'config', '.env')
print(f"Tentando carregar o arquivo .env de: {dotenv_path}")

# Carregar o arquivo .env
load_dotenv(dotenv_path)

# Testar se as variáveis foram carregadas
host = os.getenv('DB_HOST')
database = os.getenv('DB_NAME')
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')

# Verifique se as variáveis foram carregadas
print(f"Host: {host}, Database: {database}, User: {user}, Password: {password}")

Tentando carregar o arquivo .env de: c:\Users\ThiagoBizacha\Desktop\Projeto_Automacao_Coleta_Dados\tests\config\.env
Host: localhost, Database: proj_dropshipping, User: postgres, Password: admin


In [12]:
#//////////////////////// ##### CARGA BRUTA POSTGRESQL/////////////////////////////

import psycopg2
import logging 
import os

from dotenv import load_dotenv

# Caminho absoluto do arquivo .env baseado no diretório de trabalho atual
project_root = os.getcwd()
dotenv_path = os.path.join(project_root, 'config', '.env')

# Teste para garantir que as variáveis estão sendo carregadas corretamente
host = os.getenv('DB_HOST')
database = os.getenv('DB_NAME')
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')

print(f"Host: {host}, Database: {database}, User: {user}, Password: {password}")

def load_to_postgresql_bruto(df):
    """Carrega os dados do DataFrame no banco de dados PostgreSQL."""

    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    logging.info(f"Conexão com o banco de dados realizada com sucesso!")  
    print("Conexão com o banco de dados realizada com sucesso!")

    cursor = conn.cursor()

    # Iterando sobre as linhas do DataFrame e inserindo os dados na tabela
    for index, row in df.iterrows():
        cursor.execute("""
            INSERT INTO public.extract_amazon_full (
                category, rank, asin, name, title, rating, reviews, symbol, value, image, link, perc_alta, 
                classificacao_vendas, classificacao_vendas_anterior, date, origin
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
            row['category'], row['rank'], row['asin'], row['name'], row['title'], row['rating'], row['reviews'],
            row['symbol'], row['value'], row['image'], row['link'], row['perc_alta'], 
            row['classificacao_vendas'], row['classificacao_vendas_anterior'], row['date'], row['origin']
        ))

    # Confirmando as mudanças no banco de dados
    conn.commit()
    
    # Fechando a conexão com o banco de dados
    cursor.close()
    conn.close()
    
    print("Dados brutos carregados com sucesso no PostgreSQL - extract_amazon_full.")

    
load_to_postgresql_bruto(df_bruto)


Host: localhost, Database: proj_dropshipping, User: postgres, Password: admin
Conexão com o banco de dados realizada com sucesso!
Dados brutos carregados com sucesso no PostgreSQL - extract_amazon_full.


In [ ]:
### COMPILADO DADOS BRUTOS

folder_path = 'C:\\Users\\ThiagoBizacha\\Desktop\\Projeto_Automacao_Coleta_Dados\\data\\output\\bot_amazon\\extract'

def process_all_files_in_folder(folder_path):
    """Carrega todos os arquivos Excel da pasta e os carrega no PostgreSQL."""
    for file in os.listdir(folder_path):
        if file.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file)
            logging.info(f"Processando arquivo: {file_path}")
            print(f"Processando arquivo: {file_path}")

            try:
                # Ler o arquivo Excel
                df = pd.read_excel(file_path)
                logging.info(f"Arquivo {file} carregado com sucesso. Linhas: {df.shape[0]}")
                print(f"Arquivo {file} carregado com sucesso. Linhas: {df.shape[0]}")

                # Carregar o DataFrame no PostgreSQL
                load_to_postgresql_bruto(df)

            except Exception as e:
                logging.error(f"Erro ao processar o arquivo {file}: {str(e)}")
                print(f"Erro ao processar o arquivo {file}: {str(e)}")

# Executar o processamento de todos os arquivos
process_all_files_in_folder(folder_path)

## BACKUP

## V1

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

def get_amazon_bestsellers(url, retries=5, backoff_factor=0.3):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    for i in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            break
        elif response.status_code == 503:
            print(f"Failed to retrieve the page. Status code: 503. Retrying {i+1}/{retries}...")
            time.sleep(backoff_factor * (2 ** i))  # Exponential backoff
        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            return []
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page after {retries} retries.")
        return []
    
    soup = BeautifulSoup(response.content, "html.parser")
    products = []

    # Verificar a estrutura HTML da página para encontrar o seletor correto
    items = soup.select("li.a-carousel-card")  # Ajuste o seletor para os itens de produto
    print(f"Found {len(items)} items.")

    for item in items:
        # Ajuste os seletores conforme necessário
        title = item.select_one("div.p13n-sc-truncate-desktop-type2")
        price = item.select_one("span._cDEzb_p13n-sc-price_3mJ9Z")
        rating = item.select_one("span.a-icon-alt")
        image = item.select_one("img.a-dynamic-image")
        link = item.select_one("a.a-link-normal")

        # Verificação e logs dos elementos encontrados
        if title:
            title = title.get_text(strip=True)
        else:
            title = "No title"
        print(f"Title: {title}")

        if price:
            price = price.get_text(strip=True)
            # Separar o símbolo da moeda e o valor
            currency_symbol = re.findall(r'[^\d.,]+', price)[0]
            value = re.findall(r'[\d.,]+', price)[0]
        else:
            currency_symbol = "Not Available"
            value = "Not Available"
        print(f"Currency Symbol: {currency_symbol}, Value: {value}")

        if rating:
            rating = rating.get_text(strip=True)
        else:
            rating = "No rating"
        print(f"Rating: {rating}")

        if image:
            image_link = image.get('src')
        else:
            image_link = "No image link"
        print(f"Image link: {image_link}")

        if link:
            product_link = "https://www.amazon.nl" + link.get('href')
        else:
            product_link = "No product link"
        print(f"Product link: {product_link}")

        products.append({
            "title": title,
            "currency_symbol": currency_symbol,
            "price_value": value,
            "rating": rating,
            "image_link": image_link,
            "product_link": product_link
        })

    return products

def save_to_excel(products, filename):
    df = pd.DataFrame(products)
    df.to_excel(filename, index=False)
    print(f"Products saved to {filename}")

if __name__ == "__main__":
    url = "https://www.amazon.nl/gp/new-releases"
    products = get_amazon_bestsellers(url)
    save_to_excel(products, "amazon_new_releases.xlsx")


## V2

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from lxml import html

def get_category_links(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    categories = soup.select('div._p13n-zg-nav-tree-all_style_zg-browse-group__88fbz a')
    category_links = [{'category': cat.get_text(strip=True), 'link': 'https://www.amazon.nl' + cat['href']} for cat in categories]

    return category_links

def get_amazon_bestsellers(url, category, retries=5, backoff_factor=0.3):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    for i in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            break
        elif response.status_code == 503:
            print(f"Failed to retrieve the page. Status code: 503. Retrying {i+1}/{retries}...")
            time.sleep(backoff_factor * (2 ** i))  # Exponential backoff
        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            return []
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page after {retries} retries.")
        return []
    
    tree = html.fromstring(response.content)
    products = []

    items = tree.xpath('//div[@class="zg-grid-general-faceout"]')
    print(f"Found {len(items)} items in category {category}.")

    for item in items:
        #name = item.xpath('.//a[@class="a-link-normal aok-block"]/@href')
        #title = item.xpath('.//div[@class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y"]/text()')
        #brand = item.xpath('.//div[@class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y"]/text()')
        #price = item.xpath('.//span[contains(@class,"_cDEzb_p13n-sc-price_3mJ9Z") or contains(@class,"p13n-sc-price")]/text()')
        #rating = item.xpath('.//span[@class="a-icon-alt"]/text()')
        #reviews = item.xpath('.//a[@class="a-size-small a-link-normal"]/text()')
        #platform = item.xpath('.//div[@class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y"]/text()')
        #image = item.xpath('.//img[contains(@class,"a-dynamic-image") or contains(@class,"s-image")]/@src')
        #link = item.xpath('.//a[@class="a-link-normal aok-block"]/@href')

        name = item.xpath('.//a[@class="a-link-normal aok-block"]/@href')
        title = item.xpath('.//div[contains(@class, "p13n-sc-css-line-clamp-1")]/text()')
        if not title:
            title = item.xpath('.//span[contains(@class, "p13n-sc-css-line-clamp")]/text()')
        if not title:
            title = item.xpath('.//h2[contains(@class, "p13n-sc-css-line-clamp")]/text()')
        if not title:
            title = item.xpath('.//h3[contains(@class, "p13n-sc-css-line-clamp")]/text()')
        if not title:
            title = ['No title']
        brand = item.xpath('.//span[contains(@class, "p13n-sc-text")]/text()')
        price = item.xpath('.//span[contains(@class, "p13n-sc-price") or contains(@class, "p13n-sc-price-3mJ9Z")]/text()')
        rating = item.xpath('.//span[contains(@class, "a-icon-alt")]/text()')
        reviews = item.xpath('.//a[contains(@class, "a-size-small") and contains(@class, "a-link-normal")]/text()')
        platform = item.xpath('.//span[contains(@class, "p13n-sc-text")]/text()')
        image = item.xpath('.//img[contains(@class, "a-dynamic-image") or contains(@class, "s-image")]/@src')
        link = item.xpath('.//a[contains(@class, "a-link-normal") and contains(@class, "aok-block")]/@href')


        if name:
            name = name[0].split('/')[1]
        else:
            name = "No name"
        print(f"Name: {name}")

        if title:
            title = title[0].strip()
        else:
            title = "No title"
        print(f"Title: {title}")

        if brand:
            brand = brand[1].strip() if len(brand) > 1 else "No brand"
        else:
            brand = "No brand"
        print(f"Brand: {brand}")

        if price:
            price = price[0].strip()
            currency_symbol = re.findall(r'[^\d.,]+', price)[0]
            value = re.findall(r'[\d.,]+', price)[0]
        else:
            currency_symbol = "Not Available"
            value = "Not Available"
        print(f"Currency Symbol: {currency_symbol}, Value: {value}")

        if rating:
            rating = rating[0].strip()
        else:
            rating = "No rating"
        print(f"Rating: {rating}")

        if reviews:
            reviews = reviews[0].strip()
        else:
            reviews = "No reviews"
        print(f"Reviews: {reviews}")

        if platform:
            platform = platform[-1].strip()
        else:
            platform = "No platform"
        print(f"Platform: {platform}")

        if image:
            image_link = image[0]
        else:
            image_link = "No image link"
        print(f"Image link: {image_link}")

        if link:
            product_link = "https://www.amazon.nl" + link[0]
        else:
            product_link = "No product link"
        print(f"Product link: {product_link}")

        products.append({
            "name": name,
            "title": title,
            "brand": brand,
            "currency_symbol": currency_symbol,
            "price_value": value,
            "rating": rating,
            "reviews": reviews,
            "platform": platform,
            "image_link": image_link,
            "product_link": product_link,
            "category": category
        })

    return products

def save_to_excel(products, filename):
    df = pd.DataFrame(products)
    df.to_excel(filename, index=False)
    print(f"Products saved to {filename}")

if __name__ == "__main__":
    base_url = "https://www.amazon.nl/gp/bestsellers/"
    category_links = get_category_links(base_url)

    all_products = []
    for category_link in category_links:
        category = category_link['category']
        link = category_link['link']
        print(f"Processing category: {category}")
        products = get_amazon_bestsellers(link, category)
        all_products.extend(products)
        time.sleep(10)  # Adicionando tempo de espera maior entre as requisições para melhor performance

    save_to_excel(all_products, "amazon_bestsellers_by_category.xlsx")
    print("Finalizado!")


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from lxml import html

def get_category_links(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    categories = soup.select('div._p13n-zg-nav-tree-all_style_zg-browse-group__88fbz a')
    category_links = [{'category': cat.get_text(strip=True), 'link': 'https://www.amazon.nl' + cat['href']} for cat in categories]

    return category_links

def get_amazon_bestsellers(url, category, retries=5, backoff_factor=0.3):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    for i in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            break
        elif response.status_code == 503:
            print(f"Failed to retrieve the page. Status code: 503. Retrying {i+1}/{retries}...")
            time.sleep(backoff_factor * (2 ** i))  # Exponential backoff
        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            return []
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page after {retries} retries.")
        return []
    
    tree = html.fromstring(response.content)
    products = []

    items = tree.xpath('//div[@class="zg-grid-general-faceout"]')
    print(f"Found {len(items)} items in category {category}.")

    for index, item in enumerate(items, start=1):
        
        title = item.xpath('.//div[contains(@class, "p13n-sc-css-line-clamp-1")]/text()')
        if not title:
            title = item.xpath('.//span[contains(@class, "p13n-sc-css-line-clamp")]/text()')
        if not title:
            title = item.xpath('.//h2[contains(@class, "p13n-sc-css-line-clamp")]/text()')
        if not title:
            title = item.xpath('.//h3[contains(@class, "p13n-sc-css-line-clamp")]/text()')
        if not title:
            title = ['No title']
        brand = item.xpath('.//span[contains(@class, "p13n-sc-text")]/text()')
        price = item.xpath('.//span[contains(@class, "p13n-sc-price") or contains(@class, "p13n-sc-price-3mJ9Z")]/text()')
        rating = item.xpath('.//span[contains(@class, "a-icon-alt")]/text()')
        reviews = item.xpath('.//a[contains(@class, "a-size-small") and contains(@class, "a-link-normal")]/text()')
        image = item.xpath('.//img[contains(@class, "a-dynamic-image") or contains(@class, "s-image")]/@src')
        link = item.xpath('.//a[contains(@class, "a-link-normal") and contains(@class, "aok-block")]/@href')
        name = item.xpath('.//a[@class="a-link-normal aok-block"]/@href')
        if name:
            name = name[0].split('/')[1]
        else:
            name = "No name"
        print(f"Name: {name}")

        if title:
            title = title[0].strip()
        else:
            title = "No title"
        print(f"Title: {title}")

        if brand:
            brand = brand[1].strip() if len(brand) > 1 else "No brand"
        else:
            brand = "No brand"
        print(f"Brand: {brand}")

        if price:
            price = price[0].strip()
            currency_symbol = re.findall(r'[^\d.,]+', price)[0]
            value = re.findall(r'[\d.,]+', price)[0]
        else:
            currency_symbol = "Not Available"
            value = "Not Available"
        print(f"Currency Symbol: {currency_symbol}, Value: {value}")

        if rating:
            rating = rating[0].strip()
        else:
            rating = "No rating"
        print(f"Rating: {rating}")

        if reviews:
            reviews = reviews[0].strip()
        else:
            reviews = "No reviews"
        print(f"Reviews: {reviews}")

        if image:
            image_link = image[0]
        else:
            image_link = "No image link"
        print(f"Image link: {image_link}")

        if link:
            product_link = "https://www.amazon.nl" + link[0]
        else:
            product_link = "No product link"
        print(f"Product link: {product_link}")

        products.append({
            "Produto Individual": name,
            "Posição na Lista": index,
            "Imagem do Produto": image_link,
            "Título": title,
            "Link": product_link,
            "Numero de estrelas": rating,
            "Numero de avaliações": reviews,
            "Preço": f"{currency_symbol} {value}",
            "category": category
        })

    return products

def save_to_excel(products, filename):
    df = pd.DataFrame(products)
    df.to_excel(filename, index=False)
    print(f"Products saved to {filename}")

if __name__ == "__main__":
    base_url = "https://www.amazon.nl/gp/bestsellers/"
    category_links = get_category_links(base_url)

    all_products = []
    for category_link in category_links:
        category = category_link['category']
        link = category_link['link']
        print(f"Processing category: {category}")
        products = get_amazon_bestsellers(link, category)
        all_products.extend(products)
        time.sleep(10)  # Adicionando tempo de espera maior entre as requisições para melhor performance

    save_to_excel(all_products, "amazon_bestsellers_by_category.xlsx")
    print("Finalizado!")


Processing category: Amazon Renewed
Found 30 items in category Amazon Renewed.
Name: Apple-10-2-inch-Wi-Fi-Spacezwart-Refurbished
Title: No title
Brand: No brand
Currency Symbol: € , Value: 244,89
Rating: 4,4 van 5 sterren
Reviews: No reviews
Image link: https://images-eu.ssl-images-amazon.com/images/I/71F8udBqz3L._AC_UL300_SR300,200_.jpg
Product link: https://www.amazon.nl/Apple-10-2-inch-Wi-Fi-Spacezwart-Refurbished/dp/B08N89P2QZ/ref=zg_bs_g_amazon-renewed_d_sccl_1/258-7813849-1708323?psc=1
Name: Apple-iPhone-128GB-Sierra-Blue
Title: No title
Brand: No brand
Currency Symbol: € , Value: 595,00
Rating: 3,9 van 5 sterren
Reviews: No reviews
Image link: https://images-eu.ssl-images-amazon.com/images/I/61RAsVPOjxL._AC_UL300_SR300,200_.jpg
Product link: https://www.amazon.nl/Apple-iPhone-128GB-Sierra-Blue/dp/B09ML78C2J/ref=zg_bs_g_amazon-renewed_d_sccl_2/258-7813849-1708323?psc=1
Name: Apple-iPad-Air-64GB-Wi-Fi
Title: No title
Brand: No brand
Currency Symbol: € , Value: 211,89
Rating: 4,3 

KeyboardInterrupt: 

In [19]:
import requests
from lxml import html
import pandas as pd
import time
import re

def get_category_links(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return []

    tree = html.fromstring(response.content)
    categories = tree.xpath('//div[contains(@class, "_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz")]//a')
    category_links = [{'category': cat.text_content().strip(), 'link': 'https://www.amazon.nl' + cat.get('href')} for cat in categories]

    return category_links

def extract_product_details(items):
    products = []

    for index, item in enumerate(items, start=1):
        # Produto Individual (data-asin está em um elemento pai)
        product_id = item.xpath('.//@data-asin')
        product_id = product_id[0] if product_id else "No ID"
        print(f"Product ID: {product_id}")

        # Posição na Lista
        position = item.xpath('.//span[@class="zg-bdg-text"]/text()')
        position = position[0].strip() if position else str(index)
        print(f"Posição: {position}")

        # Imagem do Produto
        image = item.xpath('.//img[contains(@class, "a-dynamic-image")]/@src')
        image_link = image[0] if image else "No image link"
        print(f"Image link: {image_link}")

        # Título e Link
        title = item.xpath('.//a/span/div/text()')
        title = title[0].strip() if title else "No title"
        print(f"Title: {title}")

        link = item.xpath('.//a[contains(@class, "a-link-normal")]/@href')
        product_link = "https://www.amazon.nl" + link[0] if link else "No product link"
        print(f"Product link: {product_link}")

        # Avaliações
        rating = item.xpath('.//span[contains(@class, "a-icon-alt")]/text()')
        rating = rating[0].strip() if rating else "No rating"
        print(f"Rating: {rating}")

        reviews = item.xpath('.//span[@class="a-size-small"]/text()')
        reviews = reviews[0].strip() if reviews else "No reviews"
        print(f"Reviews: {reviews}")

        # Preço
        price = item.xpath('.//span[contains(@class, "p13n-sc-price")]/text()')
        if price:
            price = price[0].strip()
            # Separar símbolo e valor usando expressões regulares
            currency_symbol = ''.join(re.findall(r'[^\d.,]', price))
            value = ''.join(re.findall(r'[\d.,]+', price))
        else:
            currency_symbol = "Not Available"
            value = "Not Available"
        print(f"Currency Symbol: {currency_symbol}, Value: {value}")

        products.append({
            "Produto Individual": product_id,
            "Posição na Lista": position,
            "Imagem do Produto": image_link,
            "Título": title,
            "Link": product_link,
            "Numero de estrelas": rating,
            "Numero de avaliações": reviews,
            "Símbolo da Moeda": currency_symbol,
            "Valor": value
        })

    return products

def get_amazon_bestsellers(url, category, retries=5, backoff_factor=0.3):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    for i in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            break
        elif response.status_code == 503:
            print(f"Failed to retrieve the page. Status code: 503. Retrying {i+1}/{retries}...")
            time.sleep(backoff_factor * (2 ** i))  # Exponential backoff
        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            return []
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page after {retries} retries.")
        return []
    
    tree = html.fromstring(response.content)
    items = tree.xpath('//div[contains(@id, "p13n-asin-index")]')
    print(f"Found {len(items)} items in category {category}.")
    
    products = extract_product_details(items)
    return products

def save_to_excel(products, filename):
    df = pd.DataFrame(products)
    df.to_excel(filename, index=False)
    print(f"Products saved to {filename}")

if __name__ == "__main__":
    base_url = "https://www.amazon.nl/gp/bestsellers/"
    category_links = get_category_links(base_url)

    all_products = []
    for category_link in category_links:
        category = category_link['category']
        link = category_link['link']
        print(f"Processing category: {category}")
        products = get_amazon_bestsellers(link, category)
        all_products.extend(products)
        time.sleep(10)  # Adicionando tempo de espera maior entre as requisições para melhor performance

    save_to_excel(all_products, "amazon_bestsellers_by_category.xlsx")
    print("Finalizado!")


Processing category: Amazon Renewed
Found 30 items in category Amazon Renewed.
Product ID: B08N89P2QZ
Posição: #1
Image link: https://images-eu.ssl-images-amazon.com/images/I/71F8udBqz3L._AC_UL300_SR300,200_.jpg
Title: 2020 Apple iPad (10.2-inch, Wi-Fi, 32GB) Spacezwart (Refurbished)
Product link: https://www.amazon.nl/Apple-10-2-inch-Wi-Fi-Spacezwart-Refurbished/dp/B08N89P2QZ/ref=zg_bs_g_amazon-renewed_d_sccl_1/259-1305506-5100743?psc=1
Rating: 4,4 van 5 sterren
Reviews: 403
Currency Symbol: € , Value: 244,89
Product ID: B09ML78C2J
Posição: #2
Image link: https://images-eu.ssl-images-amazon.com/images/I/61RAsVPOjxL._AC_UL300_SR300,200_.jpg
Title: Apple iPhone 13 Pro, 128GB, Sierra Blue - (Refurbished)
Product link: https://www.amazon.nl/Apple-iPhone-128GB-Sierra-Blue/dp/B09ML78C2J/ref=zg_bs_g_amazon-renewed_d_sccl_2/259-1305506-5100743?psc=1
Rating: 3,9 van 5 sterren
Reviews: 403
Currency Symbol: € , Value: 595,00
Product ID: B07J4CMSVS
Posição: #3
Image link: https://images-eu.ssl-im

In [ ]:
import requests
from lxml import html
import pandas as pd
import time
import re
from datetime import datetime

def get_category_links(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return []

    tree = html.fromstring(response.content)
    categories = tree.xpath('//div[contains(@class, "_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz")]//a')
    category_links = [{'category': cat.text_content().strip(), 'link': 'https://www.amazon.nl' + cat.get('href')} for cat in categories]

    return category_links

def extract_product_details(items, category):
    products = []
    # Obter a data de hoje
    today_date = datetime.today().strftime('%Y-%m-%d')

    for index, item in enumerate(items, start=1):
        # Produto Individual (data-asin está em um elemento pai)
        product_id = item.xpath('.//@data-asin')
        product_id = product_id[0] if product_id else "No ID"
        print(f"Product ID: {product_id}")

        # Posição na Lista
        position = item.xpath('.//span[@class="zg-bdg-text"]/text()')
        position = position[0].strip() if position else str(index)
        print(f"Posição: {position}")

        # Imagem do Produto
        image = item.xpath('.//img[contains(@class, "a-dynamic-image")]/@src')
        image_link = image[0] if image else "No image link"
        print(f"Image link: {image_link}")

        # Título e Link
        title = item.xpath('.//a/span/div/text()')
        title = title[0].strip() if title else "No title"
        print(f"Title: {title}")

        link = item.xpath('.//a[contains(@class, "a-link-normal")]/@href')
        product_link = "https://www.amazon.nl" + link[0] if link else "No product link"
        print(f"Product link: {product_link}")

        # Extrair Name
        name = item.xpath('.//a[@class="a-link-normal aok-block"]/@href')
        if name:
            name = name[0].split('/')[1]
        else:
            name = "No name"
        print(f"Name: {name}")

        # Avaliações
        rating = item.xpath('.//span[contains(@class, "a-icon-alt")]/text()')
        rating = rating[0].strip() if rating else "No rating"
        print(f"Rating: {rating}")

        reviews = item.xpath('.//span[@class="a-size-small"]/text()')
        reviews = reviews[0].strip() if reviews else "No reviews"
        print(f"Reviews: {reviews}")

        # Preço
        price = item.xpath('.//span[contains(@class, "p13n-sc-price")]/text()')
        if price:
            price = price[0].strip()
            # Separar símbolo e valor usando expressões regulares
            currency_symbol = ''.join(re.findall(r'[^\d.,]', price))
            value = ''.join(re.findall(r'[\d.,]+', price))
        else:
            currency_symbol = "Not Available"
            value = "Not Available"
        print(f"Currency Symbol: {currency_symbol}, Value: {value}")

        products.append({
            "asim": product_id,
            "rank": position,
            "product_photo": image_link,
            "product_title": title,
            "link": product_link,
            "Name": name,
            "rating": rating,
            "reviews": reviews,
            "symbol": currency_symbol,
            "value": value,
            "category": category,
            "date": today_date
        })

    return products

def get_amazon_bestsellers(url, category, retries=5, backoff_factor=0.3):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    for i in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            break
        elif response.status_code == 503:
            print(f"Failed to retrieve the page. Status code: 503. Retrying {i+1}/{retries}...")
            time.sleep(backoff_factor * (2 ** i))  # Exponential backoff
        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            return []
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page after {retries} retries.")
        return []
    
    tree = html.fromstring(response.content)
    items = tree.xpath('//div[contains(@id, "p13n-asin-index")]')
    print(f"Found {len(items)} items in category {category}.")
    
    products = extract_product_details(items, category)
    return products

def save_to_excel(products, filename):
    df = pd.DataFrame(products)
    df.to_excel(filename, index=False)
    print(f"Products saved to {filename}")

if __name__ == "__main__":
    base_url = "https://www.amazon.nl/gp/bestsellers/"
    category_links = get_category_links(base_url)

    all_products = []
    for category_link in category_links:
        category = category_link['category']
        link = category_link['link']
        print(f"Processing category: {category}")
        products = get_amazon_bestsellers(link, category)
        all_products.extend(products)
        time.sleep(10)  # Adicionando tempo de espera maior entre as requisições para melhor performance

    save_to_excel(all_products, "amazon_bestsellers_by_category.xlsx")
    print("Finalizado!")


## V3

In [ ]:
import requests  # Biblioteca para fazer requisições HTTP, usada para acessar as páginas da Amazon
from lxml import html  # Biblioteca para parsing de HTML, usada para extrair informações de páginas
import pandas as pd  # Biblioteca para manipulação de dados, usada para criar e manipular DataFrames
import time  # Biblioteca para gerenciar o tempo, usada para controlar intervalos entre requisições
import re  # Biblioteca de expressões regulares, usada para manipulação de strings
from datetime import datetime  # Biblioteca para manipular datas, usada para adicionar data aos dados
import logging  # Biblioteca para criação de logs, usada para registrar eventos e erros
import random  # Biblioteca para gerar números aleatórios, usada para intervalos aleatórios entre requisições
import os  # Biblioteca para manipulação de sistemas operacionais, usada para criar diretórios

# Diretório para salvar os logs
log_directory = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/logs/"
os.makedirs(log_directory, exist_ok=True)  # Cria o diretório de logs se ele ainda não existir

# Configuração do logging para salvar no diretório especificado
logging.basicConfig(
    filename=os.path.join(log_directory, 'amazon_scraper.log'),  # Define o nome e local do arquivo de log
    level=logging.INFO,  # Define o nível de log (INFO), para registrar eventos gerais
    format='%(asctime)s - %(levelname)s - %(message)s'  # Define o formato da mensagem de log: data/hora, nível e mensagem
)

logging.info(f"Extração iniciada")  

# Função para obter os links das categorias da página inicial (Elektronica, Software ou Boeken)
def get_category_links(url):
    """
    Faz uma requisição à página principal e extrai os links das categorias.
    
    Parâmetros:
        url (str): URL da página principal da Amazon (ex: Best Sellers)

    Retorno:
        list: Lista de dicionários contendo o nome e o link completo de cada categoria
    """
    # Define o cabeçalho da requisição para simular um navegador e evitar bloqueios
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    try:
        # Faz a requisição à URL com o cabeçalho definido
        response = requests.get(url, headers=headers, timeout=10)  # Timeout de 10 segundos para evitar travamentos
        response.raise_for_status()  # Gera um erro se o status da requisição for diferente de 200 (OK)
    except requests.exceptions.RequestException as e:
        logging.error(f"Erro ao acessar a URL {url}: {e}")  # Registra o erro nos logs
        return []  # Retorna uma lista vazia caso haja falha na requisição

    # Faz o parsing do conteúdo HTML da página
    tree = html.fromstring(response.content)
    
    # Usa XPath para encontrar os links das categorias
    categories = tree.xpath('//div[contains(@class, "_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz")]//a')
    
    # Cria uma lista de dicionários contendo o nome e o link completo de cada categoria
    category_links = [{'category': cat.text_content().strip(), 'link': 'https://www.amazon.nl' + cat.get('href')} for cat in categories]

    return category_links  # Retorna a lista de categorias

# Função para extrair detalhes dos produtos de uma categoria específica
import re  # Biblioteca para expressões regulares

def extract_product_details(items, category):
    """
    Extrai detalhes dos produtos de uma categoria (ex: Best Sellers, Movers and Shakers).

    Parâmetros:
        items (list): Lista de elementos de produtos extraídos da página da categoria
        category (str): Nome da categoria de produtos

    Retorno:
        list: Lista de dicionários contendo os detalhes de cada produto
    """
    products = []  # Lista onde serão armazenados os detalhes dos produtos
    today_date = datetime.today().strftime('%Y-%m-%d')  # Data atual formatada para ser usada em cada produto

    # Itera sobre cada item da lista de produtos
    for index, item in enumerate(items, start=1):
        # Extrai o ID do produto (ASIN)
        product_id = item.xpath('.//@data-asin')
        product_id = product_id[0] if product_id else "No ID"  # Usa "No ID" caso o ID não seja encontrado

        # Extrai a posição no ranking (por exemplo, #1, #2, etc.)
        position = item.xpath('.//span[@class="zg-bdg-text"]/text()')
        position = position[0].strip() if position else str(index)  # Se a posição não estiver disponível, usa o índice do loop

        # Extrai o link da imagem do produto
        image = item.xpath('.//img[contains(@class, "a-dynamic-image")]/@src')
        image_link = image[0] if image else "No image link"  # Caso não haja imagem, usa "No image link"

        # Extrai o título do produto
        title = item.xpath('.//a/span/div/text()')
        title = title[0].strip() if title else "No title"  # Se não houver título, usa "No title"

        # Extrai o link do produto
        link = item.xpath('.//a[contains(@class, "a-link-normal")]/@href')
        product_link = "https://www.amazon.nl" + link[0] if link else "No product link"

        # Extrai o nome do produto (parte da URL)
        name = item.xpath('.//a[@class="a-link-normal aok-block"]/@href')
        name = name[0].split('/')[1] if name else "No name"

        # Extrai a classificação/avaliação do produto
        rating = item.xpath('.//span[contains(@class, "a-icon-alt")]/text()')
        rating = rating[0].strip() if rating else "No rating"

        # Extrai o número de avaliações (quantidade de reviews)
        reviews = item.xpath('.//span[@class="a-size-small"]/text()')
        reviews = reviews[0].strip() if reviews else "No reviews"

        # Extrai o preço do produto
        price = item.xpath('.//span[contains(@class, "p13n-sc-price")]/text()')
        if price:
            price = price[0].strip()
            currency_symbol = ''.join(re.findall(r'[^\d.,]', price))  # Extrai o símbolo da moeda
            value = ''.join(re.findall(r'[\d.,]+', price))  # Extrai o valor numérico do preço
        else:
            currency_symbol = "Not Available"
            value = "Not Available"

        # Extrai o número do percentual de alta (por exemplo, "427%")
        perc_alta = item.xpath('.//span[contains(@class, "zg-carousel-pct-change")]/text()')
        if perc_alta:
            perc_alta = re.search(r'\d+', perc_alta[0].strip()).group(0)
        else:
            perc_alta = "No perc alta"

        # Extrai a classificação de vendas completa (exemplo: "Classificação de vendas: 180 (anterior: 950)")
        classificacao_vendas = item.xpath('.//span[contains(@class, "zg-carousel-sales-movement")]/text()')
        classificacao_vendas = classificacao_vendas[0].strip() if classificacao_vendas else "No classificacao vendas"

        # Extrai apenas o número da classificação de vendas (antes dos parênteses) - (por exemplo, "180")
        match_vendas = re.search(r'(\d+)', classificacao_vendas)
        classificacao_vendas_numero = match_vendas.group(1) if match_vendas else "No classificacao vendas"

        # Extrai apenas o número da classificação de vendas anterior (dentro dos parênteses) - (por exemplo, "950")
        match_vendas_anterior = re.search(r'\((\d+)\)', classificacao_vendas)
        classificacao_vendas_anterior = match_vendas_anterior.group(1) if match_vendas_anterior else "No classificacao vendas anterior"

        # Adiciona os detalhes do produto à lista de produtos
        products.append({
            "category": category,
            "rank": position,
            "asin": product_id,
            "name": name,
            "title": title,
            "rating": rating,
            "reviews": reviews,
            "symbol": currency_symbol,
            "value": value,
            "image": image_link,
            "link": product_link,
            "perc_alta": perc_alta,  # Apenas o número do percentual de alta
            "classificacao_vendas": classificacao_vendas_numero,  # Apenas o número da classificação de vendas
            "classificacao_vendas_anterior": classificacao_vendas_anterior,  # Apenas o número da classificação de vendas anterior
            "date": today_date
        })

    return products  # Retorna a lista de produtos

# Função que obtém os produtos mais vendidos em uma categoria específica, com retries para evitar falhas temporárias
def get_amazon_bestsellers(url, category, retries=5, backoff_factor=1):
    """
    Faz a requisição para uma categoria específica da Amazon e coleta os produtos. Tenta novamente em caso de falhas.

    Parâmetros:
        url (str): URL da categoria da Amazon
        category (str): Nome da categoria de produtos (Best Sellers, Movers and Shakers, etc.)
        retries (int): Número máximo de tentativas de requisição
        backoff_factor (float): Fator de tempo para espera entre retries

    Retorno:
        list: Lista de produtos da categoria, ou lista vazia em caso de falha
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # Loop para realizar retries em caso de falha na requisição
    for i in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Faz a requisição com um timeout de 10 segundos
            response.raise_for_status()  # Gera um erro se o status não for 200 (OK)
            break  # Se a requisição for bem-sucedida, sai do loop
        except requests.exceptions.RequestException as e:
            logging.error(f"Erro ao acessar {url} na tentativa {i+1}/{retries}: {e}")  # Registra o erro nos logs
            time.sleep(backoff_factor * (2 ** i))  # Aumenta o tempo de espera exponencialmente (backoff)
        if i == retries - 1:
            return []  # Retorna lista vazia após o número máximo de retries

    # Faz o parsing do conteúdo da página
    tree = html.fromstring(response.content)
    
    # Extrai os produtos da categoria com base no identificador "p13n-asin-index"
    items = tree.xpath('//div[contains(@id, "p13n-asin-index")]')
    logging.info(f"Encontrados {len(items)} itens na categoria {category}.")

    # Retorna os detalhes dos produtos extraídos
    return extract_product_details(items, category)

# Função para salvar os dados consolidados em Excel
def save_to_excel_consolidated(products, directory_path):
    """
    Salva os produtos extraídos em um único arquivo Excel consolidado.

    Parâmetros:
        products (list): Lista de produtos de todas as categorias
        directory_path (str): Caminho para salvar o arquivo Excel
    """
    if not products:
        logging.warning("Nenhum produto encontrado nas categorias.")  # Registra um aviso se não houver produtos
        return

    # Cria um DataFrame a partir da lista de produtos
    df = pd.DataFrame(products)
    
    # Define o nome do arquivo com base na data atual
    today_date = datetime.today().strftime('%Y-%m-%d')
    filename = f"{directory_path}/extract_amazon_full_{today_date}.xlsx"
    
    # Salva o DataFrame em um arquivo Excel
    df.to_excel(filename, index=False)
    logging.info(f"Dados consolidados salvos em {filename}")  # Registra a conclusão da tarefa

# Função que processa uma categoria específica e adiciona a coluna "origin"
def process_category(category_url, category_type):
    """
    Processa uma categoria específica, extrai os produtos e adiciona uma coluna "origin" para identificar a origem.

    Parâmetros:
        category_url (str): URL da categoria
        category_type (str): Tipo de categoria (ex: Best Sellers, Movers and Shakers)

    Retorno:
        list: Lista de produtos com a coluna "origin" indicando a origem dos dados
    """
    # Obtém os links das subcategorias
    category_links = get_category_links(category_url)
    all_products = []

    # Processa cada subcategoria
    for category_link in category_links:
        category = category_link['category']  # Nome da subcategoria
        link = category_link['link']  # Link da subcategoria
        logging.info(f"Processando categoria: {category} - {link}")
        
        # Extrai os produtos da subcategoria
        products = get_amazon_bestsellers(link, category)
        
        # Adiciona a origem dos dados na coluna "origin"
        for product in products:
            product['origin'] = category_type  # Adiciona a origem (ex: best_sellers)
        
        all_products.extend(products)  # Adiciona os produtos à lista geral
        
        # Pausa com intervalo aleatório para evitar sobrecarga no servidor
        time.sleep(random.uniform(10, 20))

    return all_products  # Retorna a lista de produtos processados

# Função principal que coordena todo o processo
def extract_data():
    print("Extração iniciada")

    directory_path = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/output/bot_amazon"

    # Lista para armazenar os produtos de todas as categorias
    all_products = []

    # Processa Best Sellers e adiciona à lista consolidada
    base_url_bestsellers = "https://www.amazon.nl/gp/bestsellers/"
    all_products.extend(process_category(base_url_bestsellers, "best_sellers"))

    # Processa Movers and Shakers e adiciona à lista consolidada
    base_url_movers_shakers = "https://www.amazon.nl/gp/movers-and-shakers/"
    all_products.extend(process_category(base_url_movers_shakers, "movers_and_shakers"))

    # Processa New Releases e adiciona à lista consolidada
    base_url_new_releases = "https://www.amazon.nl/gp/new-releases/"
    all_products.extend(process_category(base_url_new_releases, "new_releases"))

    # Cria um DataFrame com todos os produtos
    df_consolidated = pd.DataFrame(all_products)
    
    df_consolidated['reviews'] = df_consolidated['reviews'].apply(lambda x: re.sub(r'[.,]', '', str(x)))
    df_consolidated['reviews'] = df_consolidated['reviews'].replace("No reviews", 0)  # Substituir "No reviews" por 0
    df_consolidated['reviews'] = df_consolidated['reviews'].astype(int)

    # Salva todos os dados consolidados em um arquivo Excel único
    save_to_excel_consolidated(all_products, directory_path)

    logging.info("Processo finalizado!")  # Registra a finalização do processo
    print("Base excel bruta salva")

    return df_consolidated

#if __name__ == "__main__":
 #   extract_data()


## V4 - Extract ETL

In [ ]:
import requests  # Biblioteca para fazer requisições HTTP, usada para acessar as páginas da Amazon
from lxml import html  # Biblioteca para parsing de HTML, usada para extrair informações de páginas
import pandas as pd  # Biblioteca para manipulação de dados, usada para criar e manipular DataFrames
import time  # Biblioteca para gerenciar o tempo, usada para controlar intervalos entre requisições
import re  # Biblioteca de expressões regulares, usada para manipulação de strings
from datetime import datetime  # Biblioteca para manipular datas, usada para adicionar data aos dados
import logging  # Biblioteca para criação de logs, usada para registrar eventos e erros
import random  # Biblioteca para gerar números aleatórios, usada para intervalos aleatórios entre requisições
import os  # Biblioteca para manipulação de sistemas operacionais, usada para criar diretórios

# Diretório para salvar os logs
log_directory = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/logs/"
os.makedirs(log_directory, exist_ok=True)  # Cria o diretório de logs se ele ainda não existir

# Configuração do logging para salvar no diretório especificado
logging.basicConfig(
    filename=os.path.join(log_directory, 'amazon_scraper.log'),  # Define o nome e local do arquivo de log
    level=logging.INFO,  # Define o nível de log (INFO), para registrar eventos gerais
    format='%(asctime)s - %(levelname)s - %(message)s'  # Define o formato da mensagem de log: data/hora, nível e mensagem
)

logging.info(f"Extração iniciada")  

# Função para obter os links das categorias da página inicial (Elektronica, Software ou Boeken)
def get_category_links(url, retries=5, backoff_factor=1):
    """
    Faz uma requisição à página principal e extrai os links das categorias.
    Se falhar em encontrar os links, tenta novamente até 5 vezes.

    Parâmetros:
        url (str): URL da página principal da Amazon (ex: Best Sellers)
        retries (int): Número máximo de tentativas de requisição
        backoff_factor (float): Fator de tempo para espera entre retries

    Retorno:
        list: Lista de dicionários contendo o nome e o link completo de cada categoria
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    for attempt in range(retries):
        try:
            # Faz a requisição à URL com o cabeçalho definido
            response = requests.get(url, headers=headers, timeout=10)  # Timeout de 10 segundos para evitar travamentos
            response.raise_for_status()  # Gera um erro se o status da requisição for diferente de 200 (OK)
            
            # Faz o parsing do conteúdo HTML da página
            tree = html.fromstring(response.content)
            
            # Usa XPath para encontrar os links das categorias
            categories = tree.xpath('//div[contains(@class, "_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz")]//a')
            
            if categories:
                # Cria uma lista de dicionários contendo o nome e o link completo de cada categoria
                category_links = [{'category': cat.text_content().strip(), 'link': 'https://www.amazon.nl' + cat.get('href')} for cat in categories]
                logging.info(f"Links de subcategorias encontrados para a URL: {url}")
                return category_links  # Retorna a lista de categorias
            else:
                # Se não encontrou categorias, tenta novamente
                logging.warning(f"Nenhuma subcategoria encontrada na tentativa {attempt+1}/{retries} para a URL: {url}")
                time.sleep(backoff_factor * (2 ** attempt))  # Espera exponencialmente mais a cada tentativa
        
        except requests.exceptions.RequestException as e:
            logging.error(f"Erro ao acessar a URL {url} na tentativa {attempt+1}/{retries}: {e}")
            time.sleep(backoff_factor * (2 ** attempt))  # Espera exponencialmente mais a cada tentativa
    
    logging.error(f"Não foram encontrados links de subcategorias para a URL: {url} após {retries} tentativas.")
    return []  # Retorna uma lista vazia caso haja falha na requisição após o número máximo de tentativas


# Função para extrair detalhes dos produtos de uma categoria específica
import re  # Biblioteca para expressões regulares

def extract_product_details(items, category):
    """
    Extrai detalhes dos produtos de uma categoria (ex: Best Sellers, Movers and Shakers).

    Parâmetros:
        items (list): Lista de elementos de produtos extraídos da página da categoria
        category (str): Nome da categoria de produtos

    Retorno:
        list: Lista de dicionários contendo os detalhes de cada produto
    """
    products = []  # Lista onde serão armazenados os detalhes dos produtos
    today_date = datetime.today().strftime('%Y-%m-%d')  # Data atual formatada para ser usada em cada produto

    # Itera sobre cada item da lista de produtos
    for index, item in enumerate(items, start=1):
        # Extrai o ID do produto (ASIN)
        product_id = item.xpath('.//@data-asin')
        product_id = product_id[0] if product_id else "No ID"  # Usa "No ID" caso o ID não seja encontrado

        # Extrai a posição no ranking (por exemplo, #1, #2, etc.)
        position = item.xpath('.//span[@class="zg-bdg-text"]/text()')
        position = position[0].strip() if position else str(index)  # Se a posição não estiver disponível, usa o índice do loop

        # Extrai o link da imagem do produto
        image = item.xpath('.//img[contains(@class, "a-dynamic-image")]/@src')
        image_link = image[0] if image else "No image link"  # Caso não haja imagem, usa "No image link"

        # Extrai o título do produto
        title = item.xpath('.//a/span/div/text()')
        title = title[0].strip() if title else "No title"  # Se não houver título, usa "No title"

        # Extrai o link do produto
        link = item.xpath('.//a[contains(@class, "a-link-normal")]/@href')
        product_link = "https://www.amazon.nl" + link[0] if link else "No product link"

        # Extrai o nome do produto (parte da URL)
        name = item.xpath('.//a[@class="a-link-normal aok-block"]/@href')
        name = name[0].split('/')[1] if name else "No name"

        # Extrai a classificação/avaliação do produto
        rating = item.xpath('.//span[contains(@class, "a-icon-alt")]/text()')
        rating = rating[0].strip() if rating else "No rating"

        # Extrai o número de avaliações (quantidade de reviews)
        reviews = item.xpath('.//span[@class="a-size-small"]/text()')
        reviews = reviews[0].strip() if reviews else "No reviews"

        # Extrai o preço do produto
        price = item.xpath('.//span[contains(@class, "p13n-sc-price")]/text()')
        if price:
            price = price[0].strip()
            currency_symbol = ''.join(re.findall(r'[^\d.,]', price))  # Extrai o símbolo da moeda
            value = ''.join(re.findall(r'[\d.,]+', price))  # Extrai o valor numérico do preço
        else:
            currency_symbol = "Not Available"
            value = "Not Available"

        # Extrai o número do percentual de alta (por exemplo, "427%")
        perc_alta = item.xpath('.//span[contains(@class, "zg-carousel-pct-change")]/text()')
        if perc_alta:
            perc_alta = re.search(r'\d+', perc_alta[0].strip()).group(0)
        else:
            perc_alta = 0

        # Extrai a classificação de vendas completa (exemplo: "Classificação de vendas: 180 (anterior: 950)")
        classificacao_vendas = item.xpath('.//span[contains(@class, "zg-carousel-sales-movement")]/text()')
        classificacao_vendas = classificacao_vendas[0].strip() if classificacao_vendas else "No classificacao vendas"

        # Extrai apenas o número da classificação de vendas (antes dos parênteses) - (por exemplo, "180")
        match_vendas = re.search(r'(\d+)', classificacao_vendas)
        classificacao_vendas_numero = match_vendas.group(1) if match_vendas else 0

        # Extrai apenas o número da classificação de vendas anterior (dentro dos parênteses) - (por exemplo, "950")
        match_vendas_anterior = re.search(r'\((\d+)\)', classificacao_vendas)
        classificacao_vendas_anterior = match_vendas_anterior.group(1) if match_vendas_anterior else 0

        # Adiciona os detalhes do produto à lista de produtos
        products.append({
            "category": category,
            "rank": position,
            "asin": product_id,
            "name": name,
            "title": title,
            "rating": rating,
            "reviews": reviews,
            "symbol": currency_symbol,
            "value": value,
            "image": image_link,
            "link": product_link,
            "perc_alta": perc_alta,  # Apenas o número do percentual de alta
            "classificacao_vendas": classificacao_vendas_numero,  # Apenas o número da classificação de vendas
            "classificacao_vendas_anterior": classificacao_vendas_anterior,  # Apenas o número da classificação de vendas anterior
            "date": today_date
        })

    return products  # Retorna a lista de produtos

# Função que obtém os produtos mais vendidos em uma categoria específica, com retries para evitar falhas temporárias
def get_amazon_bestsellers(url, category, retries=5, backoff_factor=1):
    """
    Faz a requisição para uma categoria específica da Amazon e coleta os produtos. Tenta novamente em caso de falhas ou
    caso nenhum produto seja encontrado (len(items) == 0).

    Parâmetros:
        url (str): URL da categoria da Amazon
        category (str): Nome da categoria de produtos (Best Sellers, Movers and Shakers, etc.)
        retries (int): Número máximo de tentativas de requisição
        backoff_factor (float): Fator de tempo para espera entre retries

    Retorno:
        list: Lista de produtos da categoria, ou lista vazia em caso de falha
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # Loop para realizar retries em caso de falha na requisição ou no parsing dos itens
    for i in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Faz a requisição com um timeout de 10 segundos
            response.raise_for_status()  # Gera um erro se o status não for 200 (OK)
            
            # Faz o parsing do conteúdo da página
            tree = html.fromstring(response.content)
            
            # Extrai os produtos da categoria com base no identificador "p13n-asin-index"
            items = tree.xpath('//div[contains(@id, "p13n-asin-index")]')
            
            # Verifica se foram encontrados itens
            if len(items) > 0:
                logging.info(f"Encontrados {len(items)} itens na categoria {category}.")
                print(f"Encontrados {len(items)} itens na categoria {category}.")
                return extract_product_details(items, category)
            else:
                logging.warning(f"Nenhum item encontrado na categoria {category} na tentativa {i+1}/{retries}.")
                print(f"Nenhum item encontrado na categoria {category} na tentativa {i+1}/{retries}.")
                time.sleep(backoff_factor * (2 ** i))  # Aumenta o tempo de espera exponencialmente (backoff)
        
        except requests.exceptions.RequestException as e:
            logging.error(f"Erro ao acessar {url} na tentativa {i+1}/{retries}: {e}")  # Registra o erro nos logs
            time.sleep(backoff_factor * (2 ** i))  # Aumenta o tempo de espera exponencialmente (backoff)
        
        # Se o número máximo de tentativas for atingido e não houver itens, retorna uma lista vazia
        if i == retries - 1:
            logging.error(f"Falha ao obter produtos da categoria {category} após {retries} tentativas.")
            print(f"Falha ao obter produtos da categoria {category} após {retries} tentativas.")
            return []  # Retorna lista vazia após o número máximo de retries

# Função para salvar os dados consolidados em Excel
def save_to_excel_consolidated(products, directory_path):
    """
    Salva os produtos extraídos em um único arquivo Excel consolidado.

    Parâmetros:
        products (list): Lista de produtos de todas as categorias
        directory_path (str): Caminho para salvar o arquivo Excel
    """
    if not products:
        logging.warning("Nenhum produto encontrado nas categorias.")  # Registra um aviso se não houver produtos
        print("Nenhum produto encontrado nas categorias.")
        return

    # Cria um DataFrame a partir da lista de produtos
    df = pd.DataFrame(products)
    
    # Define o nome do arquivo com base na data atual
    today_date = datetime.today().strftime('%Y-%m-%d')
    filename = f"{directory_path}/extract_amazon_full_{today_date}.xlsx"
    
    # Salva o DataFrame em um arquivo Excel
    df.to_excel(filename, index=False)
    logging.info(f"Dados consolidados salvos em {filename}")  # Registra a conclusão da tarefa

# Função que processa uma categoria específica e adiciona a coluna "origin"
def process_category(category_url, category_type):
    """
    Processa uma categoria específica, extrai os produtos e adiciona uma coluna "origin" para identificar a origem.

    Parâmetros:
        category_url (str): URL da categoria
        category_type (str): Tipo de categoria (ex: Best Sellers, Movers and Shakers)

    Retorno:
        list: Lista de produtos com a coluna "origin" indicando a origem dos dados
    """
    # Obtém os links das subcategorias
    category_links = get_category_links(category_url)
    
    if not category_links:
        logging.error(f"Não foram encontrados links de subcategorias para a URL: {category_url}")
        print(f"Não foram encontrados links de subcategorias para a URL: {category_url}")
        return []

    all_products = []

    # Processa cada subcategoria
    for category_link in category_links:
        category = category_link['category']  # Nome da subcategoria
        link = category_link['link']  # Link da subcategoria
        logging.info(f"Processando subcategoria: {category} - {category_type}")
        print(f"Processando subcategoria: {category} - {category_type}")
        
        # Extrai os produtos da subcategoria
        products = get_amazon_bestsellers(link, category)
        
        if not products:
            logging.warning(f"Nenhum produto encontrado na subcategoria: {category} - {category_type}")
        
        # Adiciona a origem dos dados na coluna "origin"
        for product in products:
            product['origin'] = category_type  # Adiciona a origem (ex: best_sellers)
        
        all_products.extend(products)  # Adiciona os produtos à lista geral
        
        # Pausa com intervalo aleatório para evitar sobrecarga no servidor
        time.sleep(random.uniform(10, 20))

    return all_products  # Retorna a lista de produtos processados

# Função principal que coordena todo o processo
def extract_data():
    print("Extração iniciada")

    directory_path = "C:/Users/ThiagoBizacha/Desktop/Projeto_Automacao_Coleta_Dados/data/output/bot_amazon"

    # Lista para armazenar os produtos de todas as categorias
    all_products = []

    # Processa Best Sellers e adiciona à lista consolidada
    logging.info("Iniciando processamento de Best Sellers")
    base_url_bestsellers = "https://www.amazon.nl/gp/bestsellers/"
    best_sellers_products = process_category(base_url_bestsellers, "best_sellers")
    all_products.extend(best_sellers_products)
    logging.info(f"Produtos de Best Sellers: {len(best_sellers_products)}")

    # Pausa com intervalo aleatório para evitar sobrecarga no servidor
    time.sleep(random.uniform(10, 20))

    # Processa Movers and Shakers e adiciona à lista consolidada
    logging.info("Iniciando processamento de Movers and Shakers")
    base_url_movers_shakers = "https://www.amazon.nl/gp/movers-and-shakers/"
    movers_shakers_products = process_category(base_url_movers_shakers, "movers_and_shakers")
    all_products.extend(movers_shakers_products)
    logging.info(f"Produtos de Movers and Shakers: {len(movers_shakers_products)}")

    # Pausa com intervalo aleatório para evitar sobrecarga no servidor
    time.sleep(random.uniform(10, 20))

    # Processa New Releases e adiciona à lista consolidada
    logging.info("Iniciando processamento de New Releases")
    base_url_new_releases = "https://www.amazon.nl/gp/new-releases/"
    new_releases_products = process_category(base_url_new_releases, "new_releases")
    all_products.extend(new_releases_products)
    logging.info(f"Produtos de New Releases: {len(new_releases_products)}")

    # Verifica se foram encontrados produtos
    if not all_products:
        logging.error("Nenhum produto encontrado para as categorias processadas.")
        print("Nenhum produto encontrado.")
        return

    # Cria um DataFrame com todos os produtos
    df_consolidated = pd.DataFrame(all_products)
    
    # Limpeza e formatação de reviews
    df_consolidated['reviews'] = df_consolidated['reviews'].apply(lambda x: re.sub(r'[.,]', '', str(x)))
    df_consolidated['reviews'] = df_consolidated['reviews'].replace("No reviews", 0)  # Substituir "No reviews" por 0
    df_consolidated['reviews'] = df_consolidated['reviews'].astype(int)

    # Salva todos os dados consolidados em um arquivo Excel único
    save_to_excel_consolidated(all_products, directory_path)

    logging.info("Processo finalizado!")  # Registra a finalização do processo
    print("Base excel bruta salva")

    return df_consolidated

if __name__ == "__main__":
    extract_data()
print("Finalizado")